In [ ]:
from glob import glob
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import StratifiedShuffleSplit
import itertools
from collections import defaultdict

all_model_scores = defaultdict(list)

i=0
# For every data set in the data repository...
for dataset in glob('data/car-evaluation.csv.gz'):
    # Read the data set into memory
    input_data = pd.read_csv(dataset, compression='gzip')
    #print(input_data)
    
    for dataset_repeat in range(1, 31):
        print(i)
        # Divide the data set into a training and testing sets, each time with a different RNG seed
        training_indices, testing_indices = next(iter(StratifiedShuffleSplit(input_data['class'].values,
                                                                             n_iter=1,
                                                                             train_size=0.75,
                                                                             test_size=0.25,
                                                                             random_state=dataset_repeat)))
        
        training_features = input_data.loc[training_indices].drop('class', axis=1).values
        training_classes = input_data.loc[training_indices, 'class'].values

        testing_features = input_data.loc[testing_indices].drop('class', axis=1).values
        testing_classes = input_data.loc[testing_indices, 'class'].values
        
        # These are parameters listed in the project document
        alpha_list = [0.0, 0.1, 0.25, 0.5, 0.75, 1]
        fit_prior_list = [True, False]
       
        # For every MultinomialNB parameter combination...
        for (alpha, fit_prior) in itertools.product(alpha_list, fit_prior_list):                                                                       
                                                                      
            # Create and fit the model on the training data
            clf = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
            clf.fit(training_features, training_classes)
            
            # Store the model's score with the key (name of data set, model, model parameters)
            all_model_scores[(dataset,
                              'MultinomialNB',
                              (alpha, fit_prior))].append(clf.score(testing_features,
                                                                                      testing_classes))
            
        i=i+1
print(all_model_scores)